In [67]:
import os
import random
import numpy as np

In [52]:
racism = [("africain", 757, 143), ("antisemite", 407, 8), ("arabe", 1793, 54), ("asie", 60, 263), ("chretiens", 393, 234), ("gitan", 464, 233), ("musulman", 126, 331)]
cluster_dir = "clusters/{}/"

### Récupération des tweets

In [20]:
def load_msg(raw_dataset_file):
    with open(raw_dataset_file, "r") as file:
        raw_dataset = file.read().splitlines()
    return raw_dataset

### Création d'échantillons des clusters
On fait en sorte d'avoir des types de haines équilibrés

In [40]:
def extract_random_sample(dataset, h_size, n_size):
    cd = cluster_dir.format(dataset + "_post_IF")
    dir_it = os.scandir(path=cd)
    filenames = [entry.name for entry in dir_it]
    h_filenames = [f for f in filenames if "haineux" in f]
    n_filenames = [f for f in filenames if "normal" in f]
    h_msg = []
    for h in h_filenames:
        h_msg += load_msg(cd + h)
    if (h_size < len(h_msg)):
        h_sample = random.Random(0).sample(h_msg, h_size)
    else :
        h_sample = h_msg
    n_msg = []
    for n in n_filenames:
        n_msg += load_msg(cd + n)
    if (n_size < len(n_msg)):
        n_sample = random.Random(0).sample(n_msg, n_size)
    else:
        n_sample = n_msg
    return h_sample, n_sample
    

In [53]:
racism_n_sample = []
racism_h_sample = []
for (subset, h_size, n_size) in racism:
    h_sample, n_sample = extract_random_sample(subset, h_size, n_size)
    racism_n_sample += n_sample
    racism_h_sample += h_sample

In [54]:
lgbt_h_sample, lgbt_n_sample = extract_random_sample("lgbt", 4000, 4000)

In [55]:
capa_h_sample, capa_n_sample = extract_random_sample("capacitiste", 4000, 4000)

In [56]:
miso_h_sample, miso_n_sample = extract_random_sample("miso", 4000, 4000)

In [57]:
len(racism_n_sample)

1266

In [58]:
def save_sample(sample, file):
    with open(file, "w") as f:
        for msg in sample:
            f.write(msg + '\n')

In [60]:
target_dir = "datasets/final/"
if not os.path.exists(target_dir): 
    os.mkdir(target_dir)

In [78]:
save_sample(racism_h_sample, target_dir + "racisme")
save_sample(lgbt_h_sample, target_dir + "lgbt")
save_sample(miso_h_sample, target_dir + "miso")
save_sample(capa_h_sample, target_dir + "capacitiste")

In [62]:
normal = racism_n_sample + capa_n_sample + lgbt_n_sample + miso_n_sample

### Extraire 2000 messages non haineux du premier dataset

In [65]:
with open("datasets/MillionTweets_NEFU.txt", 'r') as f:
    dataset_1M = f.read().splitlines()

In [68]:
labels_1M = np.load("datasets/MillionTweets_NEFU_labels.npy")

In [69]:
join_1M = [(msg, lab) for (msg, lab) in zip(dataset_1M, labels_1M)]

In [70]:
random.Random(0).shuffle(join_1M)

In [71]:
with open("kw_hate.txt", 'r') as f:
    hate_kw = f.read().splitlines()

In [72]:
n_max = 2000
n_norm = 0
norm_msg = []

i = 0
while (n_norm < n_max):
    msg, lab = join_1M[i]
    if (lab == 0 and not any(kw in msg for kw in hate_kw)):
        n_norm +=1
        norm_msg.append(msg)
    i += 1

In [75]:
normal += norm_msg

In [77]:
save_sample(normal, target_dir + "normal")